# Flight engine predictive maintenance
Video: https://vimeo.com/160024508

Data source: https://c3.nasa.gov/dashlink/resources/140/

## Read flight data

/data

    /Fault_X                        -> faulty equipments (X = Fan, HPC, HPT, LPC, LPT)
    /Nominal_X                      -> equipments without fault
    
        /EngineYYY                  -> engine number (3 digits)
        
            /FlightZZZ.csv          -> sensor data (sampled every second)
            /EngineHealth.csv       -> engine health data (each row = 1 flight)
            /Engine_Fuel_Effic.csv  -> engine fuel efficiency (each row = 1 flight)

First, read the data folder structure as a list of tuples:
**(path_to_folder, equipment_name (str), is_faulty (int), engine_number (int), list_of_files)**

In [ ]:
from os import walk
data_path = '/nasa/data'

f = [(dirpath.replace("\\","/"), \
      dirpath.split("/")[-2].split("_")[-1],                                 #equipment \
      True if "Fault" in dirpath.split("/")[-2] else False,                  #is fault? \
      int(dirpath.split("/")[-1].split("Engine", 1)[1]),                     #engine num
      sorted(filter(lambda fn: '.csv' in fn and 'Flight' in fn, filenames))  #csv files for flight data
     )
     for (dirpath, dirnames, filenames) in walk(data_path) \
     if ("Fault_" in dirpath or "Nominal_" in dirpath) \
     and ("Engine" in dirpath) \
     and ("#extra" not in dirpath) \
     and len(dirnames) is 0]

print f

Read flight sensor data into Pandas.DataFrame. One dataframe per engine.

In [ ]:
from pandas import read_csv
def read_file(path):
    tmp = read_csv(path)
    return tmp

In [ ]:
IS_READ_DATA = False
from pandas import concat
from time import time

st = time()
if not IS_READ_DATA:
    flight_data = []
    for (path, eq_type, is_fault, eng_num, flist) in f:
        flnum_list = []
        ditem_list = []
        for fname in flist:
            full_path = "/".join([path, fname])
            fl_num = int(fname.split('Flight',1)[1].split('.csv')[0])
            flnum_list.append(fl_num)
            ditem_list.append(read_file(full_path))
            
        this_df = concat(ditem_list, keys=flnum_list)
        del this_df['time']
        
        #keep only one index: 'fl_num'; correct 'time' and store as regular column
        this_df.reset_index(level=1, inplace=True)
        this_df.rename(columns={'level_1': 'time'}, inplace=True)
        
        #create time window column
        this_df['timew'] = (this_df['time'] // 10) * 10
        
        #append eq_type, engine_num, fault_fl_num, fault_time
        eff = read_file('/'.join([path, 'Engine_Fuel_Effic.csv']))
        eff['fl_num'] = xrange(1, eff.shape[0] + 1)
        eff.set_index(['fl_num'], inplace=True)
        
        health = read_file('/'.join([path, 'EngineHealth.csv']))
        health['fl_num'] = xrange(1, health.shape[0] + 1)
        health.set_index(['fl_num'], inplace=True)
        
        with open('/'.join([path, 'Simulation_Info.txt']), 'r') as fl:
            for ln in fl.readlines():
                if 'Fault flight number:' in ln:
                    fault_fl_num = int(ln.split('Fault flight number:',1)[1])
                elif 'Fault time:' in ln:
                    fault_time = int(ln.split('Fault time:',1)[1])
        
        flight_data.append((this_df, eq_type, eng_num, eff, (fault_fl_num, fault_time) if is_fault else None))
    
    IS_READ_DATA = True
et = time()

print 'Read data in ' + str(et-st) + 's'

# Feature engineering

Create feature **iscruise** - True if the altitude is 35000 feet

In [ ]:
from numpy import where
for dset in flight_data:
    dset[0]['iscruise'] = where(dset[0]['alt']==35000, True, False)

Group by 10-second windows and compute secondary parameters

In [ ]:
from math import sqrt
from numpy import asarray

#get pairwise correlations between all numeric input parameters (for a single flight)
def get_corrs(dset):
    a = dset.reset_index(level=0).groupby(['index','timew']).corr()
    #print 'shape a: ' + str(a.shape)
    #print a.head()
    b = a.reset_index(level=1)
    #print 'shape b: ' + str(b.shape)
    #print b.head()
    dim = a.shape[1]
    #print 'dim: ' + str(dim)
    cls = [dim*i+j for i in range(0, dim) for j in range(i+1, dim)]
    c = [b.ix[idx,1:].unstack()[cls].values.transpose() for idx in list(a.index.levels[0])]
    
    return c

In [ ]:
from numpy import number

st = time()

#select first actual dataset
corrs = []
for fld in flight_data:
    dset = fld[0].query('iscruise==True').select_dtypes(include=[number])
    corrs.append(get_corrs(dset))
    
et = time()

print 'Read data in ' + str(et-st) + 's'

In [ ]:
print corrs[0][0][0]

# Model 1: compare cruise-level parameters for two flights
All records (frequency = second) in the respective two flights are considered. <br>
Records in the first flight are marked with target = 0, others with target = 1. <br>
A classification model is fit. The accuracy of the fit indicates whether a difference can be inferred between the two samples.

In [ ]:
from numpy import array
FLIGHT_1 = 1
FLIGHT_2 = 5

#remove non-numeric fields
mod1 = equip_df.query('fl_num==' + str(FLIGHT_1) + ' or fl_num==' + str(FLIGHT_2))

mod1.ix[FLIGHT_1:FLIGHT_1,'target'] = 0
mod1.ix[FLIGHT_2:FLIGHT_2,'target'] = 1
target = mod1['target']

mod1 = mod1.drop(['time','eq','engine_num','isfault','iscruise','target'], axis=1)

In [ ]:
print 'Training records with target=0: ' + str(mod1[mod1['target']==0].shape[0])
print 'Training records with target=1: ' + str(mod1[mod1['target']==1].shape[0])
print 'Total training records: ' + str(mod1.shape[0])

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics

# fit a CART model to the data
model = DecisionTreeClassifier()
model.fit(mod1, target)

# make predictions
expected = target
predicted = model.predict(mod1)

#put target & predicted values back to 
mod1['target'] = target
mod1['predicted'] = predicted

# summarize the fit of the model
#print(metrics.classification_report(expected, predicted))
#print(metrics.confusion_matrix(expected, predicted))
print 'Accuracy: ' + str(metrics.accuracy_score(expected, predicted) * 100) + '%'

Although the accuracy of fit is high, no explanatory variables seems to correlate well with the target.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

from numpy import diag

#print mod1.columns
print mod1.corr()['predicted']
#mod1[['alt','predicted']].plot(subplots=True)

plt.matshow(diag(mod1.corr()['predicted']))

In [ ]:
#wSpec = Window.orderBy("time")
#a.select('time', fsql.lag('time', 1).over(wSpec).alias('time_lagged')).take(100)